In [1]:
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import keras

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

# import the relevant Keras modules
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD
from keras.constraints import maxnorm

import keras
print (keras.__version__)

%matplotlib inline
sns.set(style="white")

import matplotlib
matplotlib.rcParams['figure.figsize'] = [12, 8] # width, height

Using TensorFlow backend.


2.1.4


In [2]:
# Configuration 

# Data sources
include_bitcoin = 1
include_ethereum = 1
include_litecoin = 1
include_ripple = 1
include_goog_trends = 1
include_stocktwits = 0

# Data duration (training + test)
start_date_ = '2015-08-07'
end_date_ = '2018-04-18'

start_date = start_date_.replace("-", "")
end_date = end_date_.replace("-", "")

split_date = '2018-01-20'  # >10% of full dataset

# Target : Predict bitcoin (OR ethereum) daily returns OR volatility 
# target = 'btc_daily_ret'
target = 'btc_volatility'

# target = 'eth_daily_ret'
# target = 'eth_volatility'

# Explore further
# target = 'ltc_daily_ret'
# target = 'xrp_daily_ret'

coins = ['bitcoin', 'ethereum', 'litecoin', 'ripple']
coin_symbol = {'bitcoin': 'btc', 'ethereum': 'eth', 'litecoin': 'ltc', 'ripple': 'xrp'}


In [3]:
# Load dataset
dataset_df = pd.read_pickle("data/full_dataset.pkl")

In [4]:
dataset_df.head()

,Date,btc_Close,btc_Volume,eth_Close,eth_Volume,ltc_Close,ltc_Volume,xrp_Close,xrp_Volume,btc_close_off_high,...,ltc_30day_ret,xrp_close_off_high,xrp_volatility,xrp_daily_ret,xrp_7day_ret,xrp_30day_ret,btc_trends,eth_trends,ltc_trends,xrp_trends
0,2015-08-07,279.58,42484800,2.770000,164329,4.21,4192810,0.008152,363643,0.597015,...,0.036946,0.987805,0.020449,0.016459,0.016459,0.016459,29.0,62.0,69.0,78.0
1,2015-08-08,261.00,58533000,0.753325,674188,3.85,4917730,0.008476,678295,-0.969823,...,-0.087678,0.147059,0.066634,0.038217,0.038217,0.038217,33.0,35.0,42.0,64.0
2,2015-08-09,265.08,23789600,0.701897,532170,3.90,3064680,0.008808,531969,0.411945,...,0.015625,0.914530,0.041372,0.038190,0.038190,0.038190,30.0,42.0,45.0,70.0
3,2015-08-10,264.47,20979400,0.708448,405283,3.95,2239890,0.008750,472973,-0.155756,...,0.012821,-0.949686,0.018044,-0.007036,-0.007036,-0.007036,32.0,38.0,45.0,67.0
4,2015-08-11,270.39,25433900,1.070000,1463100,4.16,3426300,0.008591,282461,1.000000,...,0.053165,-1.000000,0.019998,-0.018284,-0.018284,-0.018284,29.0,40.0,48.0,59.0


In [5]:
dataset_df['eth_volatility'] = np.log(dataset_df['eth_volatility'])
dataset_df['btc_volatility'] = np.log(dataset_df['btc_volatility'])

# Benchmarks

Benchmark -I: Price-Persistent model, assumes the close price is same as the previous day. Implies that the estimated daily returns and volatility are both zero

Benchmark - II: Market-Persistent model, assumes the daily returns is same as the previous day. Implies the estimated daily returns and volatility are same as the previous day.

In [6]:
## Price-PERSISTENT MODEL: Benchmark-I

print ("Predicting:", target)

# Evaluate on test dataset
# no change from previous day => pred_bitcoin = 0 (percent change) for persistence model
actual_target = dataset_df[dataset_df['Date']>= split_date][target].values
# pred_target = 0

bm1_mae = np.mean(np.absolute((actual_target)))
bm1_mse = np.mean(np.square((actual_target)))

print("Benchmark-I: MAE", bm1_mae)
print("Benchmark-I: MSE", bm1_mse)

Predicting: btc_volatility
Benchmark-I: MAE 2.634264923378126
Benchmark-I: MSE 7.188424481804465


In [7]:
## Market-PERSISTENT MODEL: Benchmark-II

# Evaluate on test dataset
# same market behavior as previous day
pred_target = dataset_df[dataset_df['Date']>= datetime.datetime.strptime(split_date, '%Y-%m-%d') - 
                      datetime.timedelta(days=1)][target][:-1].values

bm2_mae = np.mean(np.absolute((actual_target-pred_target)))
bm2_mse = np.mean(np.square((actual_target-pred_target)))

print("Benchmark-II: MAE", bm2_mae)
print("Benchmark-II: MSE", bm2_mse)

Benchmark-II: MAE 0.4218536832328074
Benchmark-II: MSE 0.2821991334719368


In [8]:
## LSTM Begins
symbols_ = [symbol+"_" for symbol in list(coin_symbol.values())]

model_data = dataset_df
model_data.head()

,Date,btc_Close,btc_Volume,eth_Close,eth_Volume,ltc_Close,ltc_Volume,xrp_Close,xrp_Volume,btc_close_off_high,...,ltc_30day_ret,xrp_close_off_high,xrp_volatility,xrp_daily_ret,xrp_7day_ret,xrp_30day_ret,btc_trends,eth_trends,ltc_trends,xrp_trends
0,2015-08-07,279.58,42484800,2.770000,164329,4.21,4192810,0.008152,363643,0.597015,...,0.036946,0.987805,0.020449,0.016459,0.016459,0.016459,29.0,62.0,69.0,78.0
1,2015-08-08,261.00,58533000,0.753325,674188,3.85,4917730,0.008476,678295,-0.969823,...,-0.087678,0.147059,0.066634,0.038217,0.038217,0.038217,33.0,35.0,42.0,64.0
2,2015-08-09,265.08,23789600,0.701897,532170,3.90,3064680,0.008808,531969,0.411945,...,0.015625,0.914530,0.041372,0.038190,0.038190,0.038190,30.0,42.0,45.0,70.0
3,2015-08-10,264.47,20979400,0.708448,405283,3.95,2239890,0.008750,472973,-0.155756,...,0.012821,-0.949686,0.018044,-0.007036,-0.007036,-0.007036,32.0,38.0,45.0,67.0
4,2015-08-11,270.39,25433900,1.070000,1463100,4.16,3426300,0.008591,282461,1.000000,...,0.053165,-1.000000,0.019998,-0.018284,-0.018284,-0.018284,29.0,40.0,48.0,59.0


In [9]:
# LSTM Parameters
window_len = 2 # 2 # 7 # 14 # 30

training_set, test_set = model_data[model_data['Date']<split_date], \
                         model_data[model_data['Date']>=datetime.datetime.strptime(split_date, '%Y-%m-%d')-datetime.timedelta(days=window_len)]

# we don't need the date columns anymore
########### DONT'WE ?? - If we are to cross-validation, we need the dates!! or we just use the index?
training_set = training_set.drop('Date', 1)
test_set = test_set.drop('Date', 1)

In [10]:
no_of_features = training_set.shape[1]

# Columns to be normalized - Instead of traditional normalization aschemes (like MinMax on training_set), 
# normalize the columns w.r.t to the first element of the window s.t the normalized first element would be 0 
# Reference: 
norm_cols = [[symbol+metric for symbol in symbols_ for metric in ['Close','Volume']]          
             +[coin_symbol[coin]+"_trends" for coin in coins]]

norm_cols

[['btc_Close',
  'btc_Volume',
  'eth_Close',
  'eth_Volume',
  'ltc_Close',
  'ltc_Volume',
  'xrp_Close',
  'xrp_Volume',
  'btc_trends',
  'eth_trends',
  'ltc_trends',
  'xrp_trends']]

In [11]:
LSTM_training_inputs = []
for i in range(len(training_set)-window_len):
    temp_set = training_set[i:(i+window_len)].copy()
    for col in norm_cols:
        temp_set.loc[:, col] = temp_set[col]/temp_set[col].iloc[0] - 1
    LSTM_training_inputs.append(temp_set)

# model output is next price normalised to 10th previous closing price
LSTM_training_outputs = training_set[target][window_len:].values

In [12]:
LSTM_test_inputs = []
for i in range(len(test_set)-window_len):
    temp_set = test_set[i:(i+window_len)].copy()
    for col in norm_cols:
        temp_set.loc[:, col] = temp_set[col]/temp_set[col].iloc[0] - 1
    LSTM_test_inputs.append(temp_set)

LSTM_test_outputs = test_set[target][window_len:].values

print(len(training_set))
print(len(test_set))

897
91


In [13]:
# ???  easier to work with numpy arrays rather than pandas dataframes
# especially as we now only have numerical data
LSTM_training_inputs = [np.array(LSTM_training_input) for LSTM_training_input in LSTM_training_inputs]
LSTM_training_inputs = np.array(LSTM_training_inputs)

LSTM_test_inputs = [np.array(LSTM_test_inputs) for LSTM_test_inputs in LSTM_test_inputs]
LSTM_test_inputs = np.array(LSTM_test_inputs)

type(LSTM_test_inputs)
print(LSTM_test_inputs.shape, LSTM_training_inputs.shape)

(89, 2, 32) (895, 2, 32)


In [14]:
# Function to create model, required for KerasRegressor
def create_model(dropout, neurons, activ_func, optimizer):
    # use default values
    loss= 'mse'
    output_size = 1 # TODO: Can we change this to 2 or even 4 (to predict all 4 at a time - {eth, btc}_{volatility, daily_ret})
    
    print ("Start model: ", dropout, neurons)
    
    model = Sequential()

    # Add 3 layers of LSTM, followed by a dense layer
    model.add(LSTM(neurons[0], return_sequences=True, input_shape=(window_len, no_of_features)))
    model.add(Dropout(dropout))
    # from keras.layers.normalization import BatchNormalization
    # model.add(BatchNormalization())
    
    model.add(LSTM(neurons[1], return_sequences=True))
    model.add(Dropout(dropout))
    
    model.add(LSTM(neurons[2]))
    model.add(Dropout(dropout))
    
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    print ("End compilation of model: ", dropout, neurons)
        
    return model

In [15]:
# Set random seed for reproducibility
# np.random.seed(42)

print ("Train input shape: ", LSTM_training_inputs.shape)
print ("Train output shape: ", LSTM_training_outputs.shape)

# create model
model = KerasRegressor(build_fn=create_model, batch_size=4, epochs=15)

##############################################################
# grid search: dropout, neurons
dropout = [0.4] #, 0.5] # 0.6]  # [0.25, 0.5, 0.75]
neurons = [[10, 8, 4]] #, [20, 16, 8] , [40, 32, 16]] # [80, 64, 32], [160, 128, 64]
activations = ['linear'] # ['tanh', 'linear'] # , 'sigmoid']
optimizer = ['adam'] # ['sgd', 'rmsprop'] #, 'adam']
param_space = dict(dropout=dropout, neurons=neurons, activ_func=activations, optimizer=optimizer)
##############################################################

num_trials = 20
lstm_mse = []
for i in range(num_trials):
    print ("Trial: ", i)
    tseries_cv = TimeSeriesSplit(n_splits=2).split(LSTM_training_inputs)

    grid = GridSearchCV(estimator=model, param_grid=param_space, n_jobs=1, cv=tseries_cv, verbose=5)
    grid_result = grid.fit(LSTM_training_inputs, LSTM_training_outputs)
    
    bt_model = grid_result.best_estimator_
    lstm_mse.append(np.mean(np.square(bt_model.predict(LSTM_test_inputs)-(test_set[target].values[window_len:]))))

Train input shape:  (895, 2, 32)
Train output shape:  (895,)
Trial:  0
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] activ_func=linear, dropout=0.4, neurons=[10, 8, 4], optimizer=adam 
Start model:  0.4 [10, 8, 4]
End compilation of model:  0.4 [10, 8, 4]
Epoch 1/15
299/299 [==============================] - 3s 11ms/step - loss: 14.4331
Epoch 2/15
299/299 [==============================] - 0s 1ms/step - loss: 10.6915
Epoch 3/15
299/299 [==============================] - 0s 1ms/step - loss: 5.4866
Epoch 4/15
299/299 [==============================] - 0s 1ms/step - loss: 4.2578
Epoch 5/15
299/299 [==============================] - 0s 2ms/step - loss: 3.1439
Epoch 6/15
299/299 [==============================] - 0s 1ms/step - loss: 2.7730
Epoch 7/15
299/299 [==============================] - 0s 1ms/step - loss: 2.7701
Epoch 8/15
299/299 [==============================] - 0s 2ms/step - loss: 2.4189
Epoch 9/15
299/299 [==============================] - 0s 1ms/step - loss: 2

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.6s remaining:    0.0s


End compilation of model:  0.4 [10, 8, 4]
Epoch 1/15
597/597 [==============================] - 3s 6ms/step - loss: 13.2046
Epoch 2/15
597/597 [==============================] - 1s 1ms/step - loss: 4.0865
Epoch 3/15
597/597 [==============================] - 1s 1ms/step - loss: 2.9655
Epoch 4/15
597/597 [==============================] - 1s 1ms/step - loss: 2.8023
Epoch 5/15
597/597 [==============================] - 1s 1ms/step - loss: 2.9131
Epoch 6/15
597/597 [==============================] - 1s 1ms/step - loss: 2.4497
Epoch 7/15
597/597 [==============================] - 1s 1ms/step - loss: 2.5040
Epoch 8/15
597/597 [==============================] - 1s 1ms/step - loss: 2.6664
Epoch 9/15
597/597 [==============================] - 1s 1ms/step - loss: 2.7004
Epoch 10/15
597/597 [==============================] - 1s 1ms/step - loss: 2.3269
Epoch 11/15
597/597 [==============================] - 1s 1ms/step - loss: 1.9742
Epoch 12/15
597/597 [==============================] - 1s 1ms/st

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   33.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   33.6s finished


End compilation of model:  0.4 [10, 8, 4]
Epoch 1/15
895/895 [==============================] - 4s 4ms/step - loss: 7.6265
Epoch 2/15
895/895 [==============================] - 1s 2ms/step - loss: 2.8933
Epoch 3/15
895/895 [==============================] - 1s 2ms/step - loss: 2.9424
Epoch 4/15
895/895 [==============================] - 2s 2ms/step - loss: 2.2426
Epoch 5/15
895/895 [==============================] - 1s 1ms/step - loss: 2.2535
Epoch 6/15
895/895 [==============================] - 1s 1ms/step - loss: 2.2187
Epoch 7/15
895/895 [==============================] - 1s 1ms/step - loss: 1.9509
Epoch 8/15
895/895 [==============================] - 1s 1ms/step - loss: 1.8326
Epoch 9/15
895/895 [==============================] - 1s 1ms/step - loss: 1.6602
Epoch 10/15
895/895 [==============================] - 1s 1ms/step - loss: 1.6838
Epoch 11/15
895/895 [==============================] - 1s 1ms/step - loss: 1.5477
Epoch 12/15
895/895 [==============================] - 1s 1ms/ste

...

End compilation of model:  0.4 [10, 8, 4]
Epoch 1/15
895/895 [==============================] - 19s 22ms/step - loss: 8.8942
Epoch 2/15
895/895 [==============================] - 2s 2ms/step - loss: 2.7417
Epoch 3/15
895/895 [==============================] - 2s 2ms/step - loss: 2.5130
Epoch 4/15
895/895 [==============================] - 2s 2ms/step - loss: 2.3671
Epoch 5/15
895/895 [==============================] - 2s 3ms/step - loss: 2.0942
Epoch 6/15
895/895 [==============================] - 2s 3ms/step - loss: 2.1467
Epoch 7/15
895/895 [==============================] - 2s 2ms/step - loss: 2.0016
Epoch 8/15
895/895 [==============================] - 2s 2ms/step - loss: 1.8379
Epoch 9/15
895/895 [==============================] - 2s 2ms/step - loss: 1.6661
Epoch 10/15
895/895 [==============================] - 2s 2ms/step - loss: 1.7002
Epoch 11/15
895/895 [==============================] - 2s 2ms/step - loss: 1.5880
Epoch 12/15
895/895 [==============================] - 2s 2ms/s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   39.8s remaining:    0.0s


End compilation of model:  0.4 [10, 8, 4]
Epoch 1/15
597/597 [==============================] - 20s 34ms/step - loss: 12.7575
Epoch 2/15
597/597 [==============================] - 2s 3ms/step - loss: 4.9585
Epoch 3/15
597/597 [==============================] - 2s 3ms/step - loss: 3.2899
Epoch 4/15
597/597 [==============================] - 1s 2ms/step - loss: 2.9116
Epoch 5/15
597/597 [==============================] - 1s 2ms/step - loss: 2.7510
Epoch 6/15
597/597 [==============================] - 1s 2ms/step - loss: 2.8011
Epoch 7/15
597/597 [==============================] - 1s 2ms/step - loss: 2.6724
Epoch 8/15
597/597 [==============================] - 1s 2ms/step - loss: 2.4796
Epoch 9/15
597/597 [==============================] - 1s 2ms/step - loss: 2.3827
Epoch 10/15
597/597 [==============================] - 1s 2ms/step - loss: 2.3287
Epoch 11/15
597/597 [==============================] - 1s 2ms/step - loss: 2.3915
Epoch 12/15
597/597 [==============================] - 1s 2ms/

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.5min finished


End compilation of model:  0.4 [10, 8, 4]
Epoch 1/15
895/895 [==============================] - 21s 23ms/step - loss: 8.0678
Epoch 2/15
895/895 [==============================] - 2s 2ms/step - loss: 2.6760
Epoch 3/15
895/895 [==============================] - 2s 2ms/step - loss: 2.3639
Epoch 4/15
895/895 [==============================] - 2s 2ms/step - loss: 2.5245
Epoch 5/15
895/895 [==============================] - 2s 2ms/step - loss: 2.2544
Epoch 6/15
895/895 [==============================] - 2s 2ms/step - loss: 1.9883
Epoch 7/15
895/895 [==============================] - 2s 2ms/step - loss: 1.9835
Epoch 8/15
895/895 [==============================] - 2s 2ms/step - loss: 1.7966
Epoch 9/15
895/895 [==============================] - 2s 2ms/step - loss: 1.8345
Epoch 10/15
895/895 [==============================] - 2s 2ms/step - loss: 1.6787
Epoch 11/15
895/895 [==============================] - 2s 2ms/step - loss: 1.4251
Epoch 12/15
895/895 [==============================] - 2s 2ms/s

In [23]:
print(lstm_mse)
grid_result.best_params_
print(np.mean(lstm_mse), np.std(lstm_mse), np.std(lstm_mse)/np.mean(lstm_mse))

[0.22428901193980724, 0.26817932414768586, 0.2656437518009893, 0.2695021978046034, 0.23076808521677603, 0.23791901898321127, 0.22019983441396962, 0.21929012989862573, 0.21907051962315113, 0.211197158049962, 0.22484146036306968, 0.23620377823892996, 0.24870916975106047, 0.29586388072664854, 0.22435678323747898, 0.22747329911178762, 0.2521540651882372, 0.26422466644750836, 0.21037042438965298, 0.22565164755511372]
0.23879541034441348 0.02278026328391404 0.09539657085979239


In [25]:
(np.max(lstm_mse)-np.mean(lstm_mse))/np.mean(lstm_mse)

0.2389847874376039

In [17]:
print (bm1_mse)
print (bm2_mse)
print (lstm_mse)

7.188424481804465
0.2821991334719368
[0.22428901193980724, 0.26817932414768586, 0.2656437518009893, 0.2695021978046034, 0.23076808521677603, 0.23791901898321127, 0.22019983441396962, 0.21929012989862573, 0.21907051962315113, 0.211197158049962, 0.22484146036306968, 0.23620377823892996, 0.24870916975106047, 0.29586388072664854, 0.22435678323747898, 0.22747329911178762, 0.2521540651882372, 0.26422466644750836, 0.21037042438965298, 0.22565164755511372]


In [18]:
temp = create_model(0.5, [10, 8, 4], 'linear', 'adam')

temp.summary()

Start model:  0.5 [10, 8, 4]
End compilation of model:  0.5 [10, 8, 4]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_181 (LSTM)              (None, 2, 10)             1720      
_________________________________________________________________
dropout_181 (Dropout)        (None, 2, 10)             0         
_________________________________________________________________
lstm_182 (LSTM)              (None, 2, 8)              608       
_________________________________________________________________
dropout_182 (Dropout)        (None, 2, 8)              0         
_________________________________________________________________
lstm_183 (LSTM)              (None, 4)                 208       
_________________________________________________________________
dropout_183 (Dropout)        (None, 4)                 0         
_________________________________________________________________
dense

Model Robustness:

Bitcoin volatility:

[0.22428901193980724, 0.26817932414768586, 0.2656437518009893, 0.2695021978046034, 0.23076808521677603, 0.23791901898321127, 0.22019983441396962, 0.21929012989862573, 0.21907051962315113, 0.211197158049962, 0.22484146036306968, 0.23620377823892996, 0.24870916975106047, 0.29586388072664854, 0.22435678323747898, 0.22747329911178762, 0.2521540651882372, 0.26422466644750836, 0.21037042438965298, 0.22565164755511372]
0.23879541034441348 0.02278026328391404 0.09539657085979239